# TP1

## Dimitris Proios



## Requirements install 

In [ ]:
import matplotlib.pyplot as plt
import numpy as np



## Exercise 1 -- Noise and Metrics

### 1.a) Write a function that determines the Mean Squared Error (MSE) between two images x and y.

In MSE, the lower the error, the more "similar" the two images are
The two images must have the same dimension.

- MSE definition 

$ MSE = \frac{1}{N M} \sum{ _{i=1} ^{N}  \sum{ _{i=1} ^{M} (x[i,j] - y[i,j]) ^2}}$ 



In [ ]:

def mse(imageA, imageB) -> float:
	err = np.sum((imageA.astype("float") - imageB.astype("float")) ** 2)
	err /= float(imageA.shape[0] * imageA.shape[1])	
	return err


Load 2 images to test functionality

In [ ]:
from PIL import Image
imA = Image.open('./data/hdr_images/img01.tif')
imA

In [ ]:
import numpy as np
import cv2
from matplotlib import pyplot as plt

def readTifGrayScale(path):
    img = cv2.imread(path, -1)
    return img


def showTifGrayScale(img, title = ""):
    plt.imshow(img, cmap = 'gray', interpolation = 'bicubic')
    plt.xticks([]), plt.yticks([])  # to hide tick values on X and Y axis
    plt.title(title)
    plt.show()
    

In [ ]:
img1 = readTifGrayScale('./data/hdr_images/img01.tif')
img2 = readTifGrayScale('./data/hdr_images/img02.tif')
showTifGrayScale(img1)
showTifGrayScale(img2)


## 1.b)  Read in a new copy of the image cameraman.tif, keep it in its original datatype and range, i.e. uint8 and {0..255}.

In [ ]:
imgCam = readTifGrayScale('./data/cameraman.tif')
imgCam



### 1.c)  Now read in a second copy of the image cameraman.tif but map it to double and {0..1}. See Matlab im2double. Compare the two images using the MSE. Can you explain the result?

The mse is 17842.766630867263
which I believe is correct since the mse is not immune to different range.
### Notes 
- For numpy reference of types I used this documentation https://docs.scipy.org/doc/numpy-1.13.0/user/basics.types.html
- For the replaccement of im2doublle function I used https://docs.scipy.org/doc/numpy/reference/generated/numpy.interp.html 



In [ ]:
imgCam_double = np.array(imgCam).astype(np.float32)
imgCam_double = np.interp(
    imgCam_double, 
    (imgCam_double.min(), imgCam_double.max()), 
    (0, 1)
)
imgCam_double 

In [ ]:
imgCam_double.dtype 

In [ ]:
mse(imgCam_double, imgCam )


## Exercise 2 
### 2.a) Refractor the PNSR definition such that the PSNR is expressed as a function of the noise variance $σ_z ^2 $. You may assume that $σ_z ^2  = MSE(x, y)$ 

- PSNR original definition 
<br> 

(1) $ PSNR = 10 \log_{10} \frac{a^2}{MSE(x,y)} $ 
<br>

(2) $MSE(x,y) = \sigma _{z} ^2 $
<br>

(1) (2) $ PSNR = 10 \log_{10} \frac{a^2}{\sigma _{z} ^2} $ 
<br> 

 $ PSNR = 20 \log_{10} a - 10\log_{10}{\sigma _{z} ^2}  $ 
<br> 

 $ PSNR - 20 \log_{10} a  = 10\log_{10}{\sigma _{z} ^2}  $ 
<br> 

 $ \frac{PSNR - 20 \log_{10} a }{10} = \log_{10}{\sigma _{z} ^2}  $ 
<br> 

 (3) $ 10^ {\frac{PSNR - 20 \log_{10} a }{10}} = \sigma _{z} ^2  $ 

This relationship above is used in 2.b.



### 2.b) Add Gaussian noise to an image such that the PSNR ratio with the original image is 10dB, 20dB, 30dB and 40dB. Use randn, not imnoise.


In [ ]:
import math 
PSNRs = [10, 20, 30, 40]
getVarianceForPSNR = lambda db: 10**((db-20 * math.log(255, 10))/10)
varianceList = [(db, getVarianceForPSNR(db)) for db in PSNRs ]
varianceList


- Noise function:
$Z_i= N(\mu, \sigma ^2 )$


In [ ]:
def gaussianNoise(image, var, mean = 0):
      row,col = image.shape
      sigma = var**0.5
      randomGaus = np.random.normal(mean,sigma,(row,col))
      gaussNoiseMatrix = randomGaus.reshape(row,col)
      noisy = image + gaussNoiseMatrix
      return noisy, gaussNoiseMatrix

imgCam = readTifGrayScale('./data/cameraman.tif')
imgCam_double = np.array(imgCam).astype("float32")
imgCam_double = np.interp(
    imgCam_double, 
    (imgCam_double.min(), imgCam_double.max()), 
    (0, 1)
)

print("original image")
showTifGrayScale(imgCam, "original")


### 2.c) Show the noisy images on the screen. How do they look?

They look bad the more db we have 


In [ ]:
def experiment(db, variance, img):
    print("DB: ",db)
    print("variance: ", variance)
    noisedImgArray, gaussNoiseMatrix = gaussianNoise(img, variance)
    showTifGrayScale(noisedImgArray, "gaussian noise :" + str(db))
    return noisedImgArray
    
noised_images = [(db, experiment(db,var, imgCam_double )) for db,var in iter(varianceList)]


### 2.d) Show the histograms for these noisy images, can you explain what you see?
The more noisy the image the higher the difference wiht the original graph.
We see the count for every pixel value of the image.
Original image: 


In [ ]:
def convertBack(im):
    imconv= np.interp(
        im, 
        (im.min(), im.max()), 
        (0, 255)
    )
    imconv = imconv.astype("uint8")
    return imconv

def hist(img):
    imconv = convertBack(img)
    hist , bins = np.histogram(imconv.ravel(),256,[0,256])
    plt.hist(hist, bins= bins)
    plt.show()


In [ ]:
hist(imgCam_double)
from matplotlib import pyplot as plt
[ hist(im) for db, im in iter(noised_images)]


### 2.e.1) Add salt & pepper Noise to an image until the PSNR ratio between the original and the noisy image is 40 dB. 


In [ ]:
import copy 
def psnr(mseres):
    PIXEL_MAX_SQUARE = 1
    mse_square = math.sqrt(mseres )
    part1 =  math.log10(PIXEL_MAX_SQUARE / mse_square)
    return 20 * part1

imgCam = readTifGrayScale('./data/cameraman.tif')
imgCam_double = np.array(imgCam).astype("float32")
imgCam_double = np.interp(
    imgCam_double, 
    (imgCam_double.min(), imgCam_double.max()), 
    (0, 1)
)

def get_img_with_pepper_salt_noise(img, p, q):
    im = copy.deepcopy(img)
    randnums = np.random.rand(256,256)
    im[np.logical_and(randnums > p, randnums < q)] = 1
    im[ randnums <= p] = 0
    return im 

pepperedImg = get_img_with_pepper_salt_noise(imgCam_double, p=0.00015, q=0.0003)
mseNum  = mse(imgCam_double,pepperedImg)
print("mse: ", mseNum)
print("psnr: ", psnr(mseNum))
showTifGrayScale(noised_images[3][1], "gaussian image")
showTifGrayScale(pepperedImg, "Peppered image")
showTifGrayScale(imgCam_double, "Original image")


### 2.e.2) Visually compare it to the 40dB noisy image to which Gaussian noise was added. What can you conclude?
With salt and pepper noise function the picture is barely modified at 40db.
While with gaussia noise it is geavily modified.

To validate that the salt and pper nois is coorectly implemented I will try to augmenta the noise heavily.

In [ ]:
pepperedImg = get_img_with_pepper_salt_noise(imgCam_double, p=0.15, q=0.3)
mseNum  = mse(imgCam_double,pepperedImg)
print("mse: ", mseNum)
print("psnr: ", psnr(mseNum))
showTifGrayScale(pepperedImg, "Peppered image")


## Low rank approximation via SVD


### Exercise 3. Read the image peppers.png and convert it to grayscale. Perform its low-rank approximation for k = 1, ..., n. 


In [ ]:
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
imagePepperMatrix = mpimg.imread("./data/peppers.png")

In [ ]:
plt.imshow(imagePepperMatrix)


In [ ]:
imagePepperMatrix.shape

In [ ]:
import numpy as np
U, S, VT = np.linalg.svd(imagePepperMatrix)

In [ ]:
# Keep k left
U.shape

In [ ]:
# keep
S.shape

In [ ]:
VT.shape

In [ ]:
U[0][0][0]

In [ ]:

def compress_svd(image,k):
    """
    Perform svd decomposition and truncated (using k singular values/vectors) reconstruction
    returns
    --------
      reconstructed matrix reconst_matrix, array of singular values s
    """
    U,s,V = np.linalg.svd(image,full_matrices=False)
    reconst_matrix = np.dot(U[:,:k],np.dot(np.diag(s[:k]),V[:k,:]))
   
    return reconst_matrix,s


def compress_show_color_images_layer(image, k):
    """
     compress and display the reconstructed color image using the layer method 
    """
    original_shape = image.shape
    image_reconst_layers = [compress_svd(image[:,:,i],k)[0] for i in range(3)]
    image_reconst = np.zeros(image.shape)
    for i in range(3):
        image_reconst[:,:,i] = image_reconst_layers[i]
    return image_reconst

fig=plt.figure(figsize=(8, 8))
columns = 4
rows = 5
k=1
for i in range(1, columns*rows +1):
    fig.add_subplot(rows, columns, i)
    img = compress_show_color_images_layer(imagePepperMatrix,k)
    plt.imshow(img)
    k+= 1

plt.show()



### Plot the dependence between the k and MSE of k-rank approximation version of original image. Make a conclusion.

### Conclusion:
As we see in the experiment below each K component adds more of the "information" on the image.
This is consistent with the theory since the compoents include variance of the information.
This relationship is not linear since the first components are sorted to hold more of the variance observed in our data.


In [ ]:
mses = [
    (
        k,
        mse(
            imagePepperMatrix,
            compress_show_color_images_layer(imagePepperMatrix,k)
        )
    )
    for k in range(1,384)
]
plt.plot(
    [i[0] for i in mses],
    [i[1] for i in mses]
)
plt.ylabel('k')
plt.xlabel('mse')
plt.show()


### Exercise 4. 
#### 4.1)Read the image peppers.png and convert it to grayscale and add Gaussian noise N (0, 625). Perform its low-rank approximation for k = 1, ..., n. 

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import cv2 
import numpy as np
imagePepperMatrix = mpimg.imread("./data/peppers.png")
gray__imagePepperMatrix = cv2.cvtColor(
    imagePepperMatrix, 
    # cv2.COLOR_BGR2GRAY
    cv2.COLOR_RGB2GRAY
)
fig=plt.figure(figsize=(10, 10))
fig.add_subplot(1, 2, 1)
plt.title( "grayscale ")
plt.imshow(gray__imagePepperMatrix,  cmap='gray' )
noised__gray__imagePepperMatrix, noiseMatrix=  gaussianNoise(gray__imagePepperMatrix, 625, 0)
fig.add_subplot(1, 2, 2)
plt.title( "grayscale noised ")
plt.imshow(noised__gray__imagePepperMatrix,  cmap='gray' )
plt.show()

#### 4.2 Plot the dependence between the k and MSE of k-rank approximation version of original image. Make a conclusion.
The Plot the dependence between
the k and MSE of k-rank approximation version of original image. (mistake?)
I am plotting the approximation with grayscale noised image.

#### Conclusion 
The svd reconstruction ratio is maintained even with the noised image. 

In [ ]:
def grayscale_compress_svd(image,k):
    """
    Perform svd decomposition and truncated (using k singular values/vectors) reconstruction
    returns
    --------
      reconstructed matrix reconst_matrix, array of singular values s
    """
    U,s,V = np.linalg.svd(image,full_matrices=False)
    reconst_matrix = np.dot(U[:,:k],np.dot(np.diag(s[:k]),V[:k,:]))   
    return reconst_matrix,s


def grayscale_compress_show_color_images_layer(image, k):
    """
     compress and display the reconstructed color image using the layer method 
    """
    original_shape = image.shape
    image_reconst_layers = grayscale_compress_svd(image ,k)
    image_reconst = image_reconst_layers
    return image_reconst[0]

mses = [
    (
        k,
        mse(
            gray__imagePepperMatrix,
            grayscale_compress_show_color_images_layer(gray__imagePepperMatrix,k)
        )
    )
    for k in range(1,384)
]
plt.plot(
    [i[0] for i in mses],
    [i[1] for i in mses]
)
plt.ylabel('k')
plt.xlabel('mse')
plt.show()


### Noise Visibility Function (NVF)
${NVF = \frac{1}{1 + \theta \sigma _{x} ^{2} (i, j)}}$
<br>
${\theta = \frac{D}{ \sigma _{x_{max}} ^{2} }}$

### Exercise 5. ex
(a) Read the image lena.png and convert it to grayscale.

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import cv2 
import numpy as np
imageLena = mpimg.imread("./data/lena.png")
gray__imageLena = cv2.cvtColor(
    imageLena, 
    cv2.COLOR_RGB2GRAY
    
)
plt.imshow(gray__imageLena,  cmap='gray' )
(h, w) = gray__imageLena.shape[:2] 




(b) Add a watermark to the image with and without applying NVF function the different
values of ${σ ^z _2}$ (10, 25, 50, 75) and D. 
<br>
Choose the window size appropriate to used image.

<br>

What can you say about the impact of NVF function?

#### Solution 
I am confused if the watermarking means to put an image on top of the other with transparency,  or to hide data in the other image.
TODO https://www.pyimagesearch.com/2016/04/25/watermarking-images-with-opencv-and-python/

Fistly after defining the noise visibillity function, 
I create a grayscale watermark image.
Then I add a another dimension for the alpha transparency parameter.

In [ ]:
def theta(D, maxLocalVar):
    return 

def NVF(i, j, theta, variance_matrix):
    return 1/(1+ theta* variance_matrix[i][j])

def embeddingEquation(i,j, NVF, x, z):
    return x[i][j] + (1-NVF) * z[i][j]

watermark = cv2.imread("./data/watermark.png", cv2.IMREAD_UNCHANGED)
gray__watermark  = cv2.cvtColor(
    watermark, 
    cv2.COLOR_RGB2GRAY
)
(wH, wW) = gray__watermark.shape[:2]
gray__watermark = gray__watermark.astype("float32")


In [ ]:
arr = np.ones((wH, wW)) * 255
gray_tansp = np.dstack([gray__watermark, arr ])
gray_tansp

In [ ]:
output = gray__imageLena.copy()
h, w = output.shape[:2]
plt.imshow(gray__watermark,  cmap='gray')
overlay = np.zeros((h, w), dtype="uint8")
overlay[h - wH:h, w - wW:w] = gray__watermark
overlay = overlay.astype("float32")
# overlay, output
plt.imshow(overlay,  cmap='gray')
plt.show()
plt.imshow(output,  cmap='gray')
plt.show()
print(output) 
# cv2.addWeighted(overlay, 0.25, output, 1.0, 0, output)
print(output) 
plt.imshow(output,  cmap='gray')
plt.show()
# gray__watermark.shape, output.shape


(c) Report the dependency between the parameters ${σ ^z _2}$ , D and original image.




Exercise 6. ex
(a) You are given a set of images hdr images. Combine the images (not necessary all) to one
image in such a way that the result image has higher quality then all given images in the
set. You can sum, subtract the images, divide by some constant, multiply by some mask,
etc.

(b) Visualise the results and explain how did you obtain them.

Solution for a, b
Base on this example:
TODO https://docs.opencv.org/4.1.0/d2/df0/tutorial_py_hdr.html

In [ ]:
import numpy as np
import cv2 
import matplotlib.pyplot as plt

img_fn= ["./data/hdr_images/"+f for f in ["img01.tif", "img02.tif", "img03.tif", "img04.tif", "img05.tif"]]
img_list = [cv2.imread(fn, -1) for fn in img_fn]
exposure_times = np.array([0.0333, 0.25, 1.0, 2.5, 15.0], dtype=np.float32)

#Merge exposures into HDR image
merge_debevec = cv2.createMergeDebevec()
hdr_debevec = merge_debevec.process(img_list, times=exposure_times.copy())
merge_robertson = cv2.createMergeRobertson()
hdr_robertson = merge_robertson.process(img_list, times=exposure_times.copy())

# Tonemap HDR image
tonemap1 = cv2.createTonemap(gamma=2.2)
res_debevec = tonemap1.process(hdr_debevec.copy())

# Exposure fusion using Mertens
merge_mertens = cv2.createMergeMertens()
res_mertens = merge_mertens.process(img_list)
# Convert datatype to 8-bit and save
res_debevec_8bit = np.clip(res_debevec*255, 0, 255).astype('uint8')
# res_robertson_8bit = np.clip(res_robertson*255, 0, 255).astype('uint8')
res_mertens_8bit = np.clip(res_mertens*255, 0, 255).astype('uint8')
cv2.imwrite("ldr_debevec.jpg", res_debevec_8bit)
cv2.waitKey(0)
cv2.destroyAllWindows() #close the image window
img1= cv2.imread("ldr_debevec.jpg")
rgb = cv2.cvtColor(img1, cv2.COLOR_BGR2RGB)
plt.imshow(rgb, cmap = plt.cm.Spectral)
plt.show()
cv2.imwrite("fusion_mertens.jpg", res_mertens_8bit)
cv2.waitKey(0)
cv2.destroyAllWindows() #close the image window
img2= cv2.imread("fusion_mertens.jpg")
rgb = cv2.cvtColor(img2, cv2.COLOR_BGR2RGB)
plt.imshow(rgb, cmap = plt.cm.Spectral)
plt.show()



RENOIR
### You are given a ”renoir” set of two images (reference and noisy) from the RENOIR dataset 2 .
### Exercise 7. ex
(a) Visualise all color channels of both images. 
Are the all channels equally affected by the noise?
<br>

In [ ]:
import scipy.ndimage as ndimage
from skimage.exposure import histogram

im1 = ndimage.imread("./data/renoir/Noisy.bmp")
im2 = ndimage.imread("./data/renoir/Reference.bmp", -1)

plt.imshow(im1)
plt.show()
plt.imshow(im2)
plt.show()

red_images = im1[:,:,0]
green_images = im1[:,:,1]
blue_images = im1[:,:,2]

In [ ]:

histogram(red_images)

In [ ]:

plt.plot(
    np.ndarray.flatten(green_images)
)
plt.show()

In [ ]:
plt.plot(
    np.ndarray.flatten(blue_images)
)
plt.show()
    
plt.ylabel('pixel ')
plt.xlabel('value ')
plt.show()
plt.plot(
    np.ndarray.flatten(im2)
)
plt.show()



(b) Try to decrease the noise via image down/up sampling.
<br>

(a) Do it for the RGB image. Measure the PSNR between the reference and de-noised
images 3 .
<br>

(b) Do it for the grayscale image. Measure the PSNR between the grayscale reference
and de-noised images.
<br>

(c) Do the denoising for the RGB image and after convert it to the grayscale. Measure
the PSNR. Does the obtained result is different from the (b)? Explain the r
<br>


Exercise 8. You are given a set of 4 images: tp1 101.png - tp1 104.png. For one of these
images perform the segmentation of the text information. See the example in Figure 3. Some
graphical elements can be segmented as well.


In [ ]:
#TODO https://scikit-image.org/docs/dev/auto_examples/edges/plot_edge_filter.html
import numpy as np
import matplotlib.pyplot as plt

from skimage.filters import gaussian, frangi
image = cv2.imread("./data/tp1_101.png")
image.shape
edge_gaussian = gaussian(image)
edge_frangi = frangi(image)
fig, ax = plt.subplots(ncols=3, sharex=True, sharey=True,
                       figsize=(8, 4))
ax[0].imshow(image)
ax[0].set_title('original image ')
ax[1].imshow(edge_gaussian)
ax[1].set_title('Gaussian Edge Detection')
ax[2].imshow(edge_frangi)
ax[2].set_title('frangi Edge Detection')


for a in ax:
    a.axis('off')

plt.tight_layout()
plt.show()


## Nuts and bolts
In this little project you will design and test a program that can recognize various nuts and
bolts in an image using Matlab’s morphological functions and a bit of statistics. The image can
be seen in Figure 4. Matlab contains an excellent tutorial segmenting and counting rice in an
image, which you can work through as preparation.
The principle steps that need to be done are the following:
• Segment the foreground which contains all parts, from the background. You can use
4
http://www.mathworks.ch/ch/help/images/image-enhancement-and-analysis.html
morphological opening, e.g. imopen to ascertain background statistics or use the so called
Otsi’s method implemented by Matlab graythresh.
• Use morphology to remove any noise from the image
• Select all individual items using Matlab’s bwlabel and bwconncomp.
• To gather statistics deploy Matlab’s regionprops function. It is capable of collecting a
vast amount of information on binary objects which in term can be used to distinguish
the various parts from each other.
• Find a combination of metrics to separate the different parts as best as possible.

### Exercise 9. ex
1. Implement the image segmentation and statistics gathering functions
2. Report on what statistics work and why (not).